In [193]:
# ========== 1. CONFIGURACIÓN DEL ENTORNO PERSONAL ==========
%pip install google-generativeai pypdf pandas matplotlib seaborn tabulate openpyxl -q

import pandas as pd
import numpy as np
import os
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo visual personal
plt.style.use('default')
sns.set_palette("husl")

# Crear estructura de datos personal
print("🏠 Configurando tu espacio personal de HealthGenie...")

# Crear directorio personal para datos
personal_data_dir = "mi_salud_personal"
if not os.path.exists(personal_data_dir):
    os.makedirs(personal_data_dir)
    print(f"✓ Directorio personal creado: {personal_data_dir}")

# Archivo de perfil personal
perfil_file = os.path.join(personal_data_dir, "mi_perfil.json")

# Cargar base de datos de alimentos
try:
    with open('alimentos.json', 'r', encoding='utf-8') as f:
        alimentos_db = json.load(f)
    
    # Convertir a DataFrame para mejor manejo
    df_alimentos = pd.DataFrame(alimentos_db['alimentos'])
    print(f"✓ Base de datos de alimentos cargada: {len(df_alimentos)} alimentos disponibles")
    
    # Mostrar categorías disponibles
    categorias = df_alimentos['categoria'].unique()
    print(f"✓ Categorías disponibles: {', '.join(categorias)}")
    
except Exception as e:
    print(f"❌ Error cargando base de datos de alimentos: {e}")
    df_alimentos = pd.DataFrame()

# Guías nutricionales basadas en la base de datos
guias_nutricionales = {
    'general': [
        "Elegir alimentos con bajo índice glucémico (IG < 55) para mejor control de glucosa.",
        "Priorizar alimentos ricos en fibra para mejorar saciedad y digestión.",
        "Combinar proteínas magras con carbohidratos complejos.",
        "Incluir grasas saludables como aguacate, frutos secos y aceite de oliva.",
        "Mantener porciones controladas según las características nutricionales."
    ],
    'diabetes': [
        "Evitar alimentos con carga glucémica alta (>20).",
        "Preferir frutas con IG bajo como fresas, arándanos y manzanas.",
        "Elegir carbohidratos integrales sobre refinados.",
        "Incluir legumbres por su bajo IG y alto contenido de fibra.",
        "Monitorear carbohidratos totales por comida."
    ],
    'hipertension': [
        "Limitar alimentos procesados altos en sodio.",
        "Incluir frutas y verduras ricas en potasio.",
        "Elegir lácteos bajos en grasa.",
        "Usar aceite de oliva en lugar de grasas saturadas.",
        "Controlar porciones de grasas totales."
    ],
    'perdida_peso': [
        "Priorizar alimentos de baja densidad calórica.",
        "Incluir verduras de hoja verde en cada comida.",
        "Elegir proteínas magras como pollo sin piel y pescados magros.",
        "Controlar porciones de grasas y carbohidratos.",
        "Aumentar consumo de fibra para mayor saciedad."
    ]
}

# Guardar guías nutricionales
guias_file = os.path.join(personal_data_dir, "guias_nutricionales.json")
with open(guias_file, 'w', encoding='utf-8') as f:
    json.dump(guias_nutricionales, f, ensure_ascii=False, indent=2)

print("✓ Estructura de datos personal configurada")
print("✓ Base de guías nutricionales cargada")

# Importar librerías necesarias
import google.generativeai as genai
from IPython.display import Markdown, display
import getpass
print("✓ Librerías importadas exitosamente")

Note: you may need to restart the kernel to use updated packages.
🏠 Configurando tu espacio personal de HealthGenie...
✓ Base de datos de alimentos cargada: 97 alimentos disponibles
✓ Categorías disponibles: carbohidratos, proteinas, verduras, frutas, lacteos, grasas, bebidas, procesados
✓ Estructura de datos personal configurada
✓ Base de guías nutricionales cargada
✓ Librerías importadas exitosamente



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [194]:
# ========== 2. CONFIGURACIÓN PERSONAL DE API ==========

def get_api_key():
    """Obtiene tu clave API personal de Google AI"""
    # Intentar variable de entorno primero
    api_key = os.getenv('GOOGLE_API_KEY')
    if api_key:
        return api_key
    
    # Intentar archivo de configuración personal
    config_file = os.path.join(personal_data_dir, "config.json")
    if os.path.exists(config_file):
        try:
            with open(config_file, 'r') as f:
                config = json.load(f)
                if 'api_key' in config:
                    return config['api_key']
        except:
            pass
    
    # Solicitar clave API
    print("🔑 Configuración de tu acceso personal a HealthGenie")
    print("📝 Necesitas tu clave API de Google AI")
    print("🔗 Obtén una gratis en: https://makersuite.google.com/app/apikey")
    print("💾 Se guardará de forma segura en tu dispositivo")
    
    api_key = getpass.getpass("Ingresa tu clave API de Google AI: ")
    
    # Guardar para uso futuro
    if api_key:
        save_key = input("¿Guardar clave para próximas sesiones? (s/n): ").lower() == 's'
        if save_key:
            config = {'api_key': api_key}
            with open(config_file, 'w') as f:
                json.dump(config, f)
            print("✓ Clave guardada de forma segura")
    
    return api_key

# Configurar tu acceso personal
try:
    api_key = get_api_key()
    if api_key:
        genai.configure(api_key=api_key)
        print("✅ Tu acceso a HealthGenie está configurado")
    else:
        print("⚠️ Sin acceso a IA. Algunas funciones estarán limitadas")
except Exception as e:
    print(f"❌ Error en configuración: {e}")

# Modelos personalizados para tu experiencia
MODEL_NAME = "models/gemini-2.0-flash"
EMBEDDING_MODEL = "models/text-embedding-004"

print(f"🤖 Tu modelo personal: {MODEL_NAME}")
print(f"🧠 Sistema de memoria: {EMBEDDING_MODEL}")

# Cargar tus guías de recetas y tips
try:
    with open(guias_file, 'r', encoding='utf-8') as f:
        mis_recetas_tips = json.load(f)
    
    # Crear lista completa de guías para tu contexto
    todas_mis_guias = []
    for categoria, guias in mis_recetas_tips.items():
        todas_mis_guias.extend(guias)
    
    print(f"✓ {len(todas_mis_guias)} guías de recetas y tips cargadas para tu perfil")
    
except Exception as e:
    print(f"⚠️ Error cargando guías: {e}")
    todas_mis_guias = [
        "Mantener hábitos saludables de alimentación.",
        "Preparar comidas caseras nutritivas.",
        "Incluir variedad de ingredientes frescos."
    ]

✅ Tu acceso a HealthGenie está configurado
🤖 Tu modelo personal: models/gemini-2.0-flash
🧠 Sistema de memoria: models/text-embedding-004
✓ 20 guías de recetas y tips cargadas para tu perfil


In [195]:
# ========== 3. TU PERFIL BÁSICO ==========

class MiPerfilBasico:
    """Gestiona tu perfil básico para recetas personalizadas"""
    
    def __init__(self):
        self.archivo_perfil = perfil_file
        self.perfil = self.cargar_perfil()
    
    def cargar_perfil(self):
        """Carga tu perfil existente o crea uno nuevo"""
        if os.path.exists(self.archivo_perfil):
            try:
                with open(self.archivo_perfil, 'r', encoding='utf-8') as f:
                    perfil = json.load(f)
                print("✓ Tu perfil personal cargado")
                # Validar alimentos frecuentes
                perfil = self.validar_alimentos_frecuentes(perfil)
                return perfil
            except:
                pass
        
        # Crear perfil nuevo
        print("👋 ¡Hola! Vamos a crear tu perfil básico para recetas personalizadas")
        return self.crear_perfil_nuevo()
    
    def validar_alimentos_frecuentes(self, perfil):
        """Valida que los alimentos frecuentes existan en la base de datos"""
        if 'alimentosFrecuentes' not in perfil or not perfil['alimentosFrecuentes']:
            return perfil
        
        if df_alimentos.empty:
            print("⚠️ Base de datos de alimentos no disponible para validación")
            return perfil
        
        alimentos_validos = df_alimentos['nombre'].tolist()
        alimentos_frecuentes_originales = perfil['alimentosFrecuentes'].copy()
        alimentos_frecuentes_validados = []
        
        for alimento in alimentos_frecuentes_originales:
            if alimento in alimentos_validos:
                alimentos_frecuentes_validados.append(alimento)
            else:
                # Buscar alimento similar usando embeddings si están disponibles
                alimento_similar = self.buscar_alimento_similar(alimento, alimentos_validos)
                if alimento_similar:
                    print(f"🔄 '{alimento}' → '{alimento_similar}' (corregido automáticamente)")
                    alimentos_frecuentes_validados.append(alimento_similar)
                else:
                    print(f"❌ '{alimento}' no encontrado en la base de datos y se removió")
        
        perfil['alimentosFrecuentes'] = alimentos_frecuentes_validados
        
        if len(alimentos_frecuentes_validados) != len(alimentos_frecuentes_originales):
            print(f"✓ Alimentos frecuentes validados: {len(alimentos_frecuentes_validados)}/{len(alimentos_frecuentes_originales)}")
            self.guardar_perfil(perfil)
        
        return perfil
    
    def buscar_alimento_similar(self, alimento_buscado, alimentos_validos):
        """Busca un alimento similar usando embeddings de IA"""
        # Si no hay embeddings disponibles, usar búsqueda simple por texto
        if not alimentos_embeddings_cache:
            return self.busqueda_textual_simple(alimento_buscado, alimentos_validos)
        
        try:
            # Usar el sistema de embeddings para encontrar el alimento más similar
            resultado = encontrar_alimentos_recomendados(
                consulta=alimento_buscado,
                perfil={'esDiabetico': False, 'tipoDiabetes': None},  # Perfil neutro para validación
                top_k=1
            )
            
            if not resultado.empty:
                alimento_encontrado = resultado.iloc[0]['nombre']
                # Verificar que la similitud sea razonable (>0.5)
                if resultado.iloc[0].get('similitud', 0) > 0.5:
                    return alimento_encontrado
            
            return None
            
        except Exception as e:
            print(f"⚠️ Error en búsqueda con embeddings: {e}")
            return self.busqueda_textual_simple(alimento_buscado, alimentos_validos)
    
    def busqueda_textual_simple(self, alimento_buscado, alimentos_validos):
        """Búsqueda simple por coincidencias de texto como fallback"""
        alimento_lower = alimento_buscado.lower()
        
        # Buscar coincidencias parciales
        for alimento_valido in alimentos_validos:
            if (alimento_lower in alimento_valido.lower() or 
                alimento_valido.lower() in alimento_lower):
                return alimento_valido
        
        return None
    
    def crear_perfil_nuevo(self):
        """Crea tu perfil personal por primera vez"""
        print("\n📋 Información básica para personalizar tus recetas")
        
        # Información básica
        nombre = input("Tu nombre (opcional, solo para personalización): ").strip()
        if not nombre:
            nombre = "Usuario"
        
        # Información sobre diabetes
        print("\n🩺 Información médica (para recomendaciones seguras)")
        es_diabetico_input = input("¿Eres diabético? (s/n): ").strip().lower()
        es_diabetico = es_diabetico_input in ['s', 'sí', 'si', 'yes', 'y']
        
        tipo_diabetes = None
        if es_diabetico:
            print("Tipos de diabetes:")
            print("1. Tipo 1 (dependiente de insulina)")
            print("2. Tipo 2 (puede requerir insulina)")
            print("3. Gestacional")
            print("4. Otro/No especificado")
            
            tipo_input = input("Selecciona tu tipo de diabetes (1-4): ").strip()
            tipo_map = {
                '1': 'tipo1',
                '2': 'tipo2', 
                '3': 'gestacional',
                '4': 'otro'
            }
            tipo_diabetes = tipo_map.get(tipo_input, 'tipo2')  # Default tipo 2
        
        # Preferencias alimentarias
        print("\n🍽️ Preferencias alimentarias (opcional)")
        restricciones = input("¿Tienes otras restricciones alimentarias? (ej: vegetariano, sin gluten, sin lácteos): ").strip()
        if not restricciones:
            restricciones = "Ninguna"
        
        # Si es diabético, agregar automáticamente control glucémico
        if es_diabetico:
            if restricciones == "Ninguna":
                restricciones = "Diabetes"
            else:
                restricciones += ", Diabetes"
        
        # Tipo de cocina preferida
        cocina_preferida = input("¿Qué tipo de cocina prefieres? (ej: mediterránea, asiática, mexicana): ").strip()
        if not cocina_preferida:
            cocina_preferida = "Variada"
        
        # Alimentos frecuentes con validación
        print("\n🍽️ Alimentos que consumes frecuentemente")
        print("💡 Estos ayudarán a personalizar tus recomendaciones")
        self.mostrar_categorias_disponibles()
        
        alimentos_frecuentes = self.solicitar_alimentos_frecuentes()
        
        perfil = {
            'nombre': nombre,
            'esDiabetico': es_diabetico,
            'tipoDiabetes': tipo_diabetes,
            'restricciones': restricciones,
            'cocina_preferida': cocina_preferida,
            'alimentosFrecuentes': alimentos_frecuentes,
            'fecha_creacion': datetime.now().isoformat(),
            'fecha_actualizacion': datetime.now().isoformat()
        }
        
        self.guardar_perfil(perfil)
        print(f"\n✅ ¡Perfil creado para {nombre}!")
        if es_diabetico:
            print(f"🩺 Configurado para diabetes {tipo_diabetes}")
            print("🎯 Se priorizarán alimentos con IG ≤ 55 y CG ≤ 10")
        print("🍳 Ahora podrás recibir recetas y tips personalizados")
        
        return perfil
    
    def mostrar_categorias_disponibles(self):
        """Muestra las categorías de alimentos disponibles"""
        if df_alimentos.empty:
            return
        
        print("📋 Categorías disponibles:")
        categorias = df_alimentos.groupby('categoria')['nombre'].count()
        for categoria, count in categorias.items():
            ejemplos = df_alimentos[df_alimentos['categoria'] == categoria]['nombre'].head(3).tolist()
            print(f"  • {categoria.title()}: {', '.join(ejemplos)}... ({count} opciones)")
    
    def solicitar_alimentos_frecuentes(self):
        """Solicita y valida alimentos frecuentes del usuario"""
        alimentos_frecuentes = []
        
        print("\nIngresa hasta 4 alimentos que consumas frecuentemente:")
        print("(Presiona Enter sin escribir nada para terminar)")
        
        for i in range(4):
            alimento = input(f"Alimento {i+1}: ").strip()
            if not alimento:
                break
            
            # Validar que existe en la base de datos
            if not df_alimentos.empty:
                alimentos_validos = df_alimentos['nombre'].tolist()
                if alimento in alimentos_validos:
                    alimentos_frecuentes.append(alimento)
                    print(f"✓ '{alimento}' agregado")
                else:
                    alimento_similar = self.buscar_alimento_similar(alimento, alimentos_validos)
                    if alimento_similar:
                        confirmar = input(f"¿Te refieres a '{alimento_similar}'? (s/n): ").strip().lower()
                        if confirmar in ['s', 'sí', 'si', 'yes', 'y']:
                            alimentos_frecuentes.append(alimento_similar)
                            print(f"✓ '{alimento_similar}' agregado")
                        else:
                            print(f"❌ '{alimento}' no encontrado")
                    else:
                        print(f"❌ '{alimento}' no encontrado en la base de datos")
                        print("💡 La IA buscará automáticamente alimentos similares")
            else:
                # Si no hay base de datos, agregar sin validar
                alimentos_frecuentes.append(alimento)
        
        if not alimentos_frecuentes:
            # Alimentos por defecto de la base de datos
            print("📝 Usando alimentos frecuentes por defecto...")
            alimentos_frecuentes = ["Arroz integral cocido", "Pollo pechuga sin piel", "Espinacas", "Manzana"]
        
        return alimentos_frecuentes
    
    def guardar_perfil(self, perfil=None):
        """Guarda tu perfil"""
        if perfil is None:
            perfil = self.perfil
        
        perfil['fecha_actualizacion'] = datetime.now().isoformat()
        
        with open(self.archivo_perfil, 'w', encoding='utf-8') as f:
            json.dump(perfil, f, ensure_ascii=False, indent=2)
        
        self.perfil = perfil
        print("✓ Tu perfil ha sido guardado")
    
    def mostrar_perfil(self):
        """Muestra tu perfil actual"""
        print(f"\n👤 === Tu Perfil Básico ===")
        print(f"📛 Nombre: {self.perfil.get('nombre', 'Usuario')}")
        print(f"🩺 Diabético: {'Sí' if self.perfil.get('esDiabetico', False) else 'No'}")
        if self.perfil.get('esDiabetico', False):
            print(f"🩺 Tipo de diabetes: {self.perfil.get('tipoDiabetes', 'No especificado')}")
        print(f"🚫 Restricciones: {self.perfil.get('restricciones')}")
        print(f"🍽️ Cocina preferida: {self.perfil.get('cocina_preferida')}")

# Inicializar tu perfil personal
mi_perfil = MiPerfilBasico()
mi_perfil.mostrar_perfil()

✓ Tu perfil personal cargado

👤 === Tu Perfil Básico ===
📛 Nombre: Santiago
🩺 Diabético: Sí
🩺 Tipo de diabetes: tipo2
🚫 Restricciones: Diabetes
🍽️ Cocina preferida: Variada


In [196]:
# ========== 4. SISTEMA DE RECOMENDACIONES NUTRICIONALES ==========

# Variable global para embeddings de alimentos
alimentos_embeddings_cache = None

def crear_embeddings_alimentos(alimentos_textos, task_type="retrieval_document"):
    """Crea embeddings para los alimentos de la base de datos"""
    global alimentos_embeddings_cache

    if not isinstance(alimentos_textos, list) or not all(isinstance(text, str) for text in alimentos_textos):
        print("❌ La entrada debe ser una lista de strings")
        return None
    if not alimentos_textos:
        print("⚠️ Lista de alimentos vacía")
        return []

    try:
        result = genai.embed_content(
            model=EMBEDDING_MODEL,
            content=alimentos_textos,
            task_type=task_type
        )
        print(f"✓ Generados {len(result.get('embedding', []))} embeddings de alimentos")
        return result.get('embedding')
    except Exception as e:
        print(f"❌ Error generando embeddings de alimentos: {e}")
        return None

def crear_descripcion_alimento(row):
    """Crea una descripción textual de un alimento para embeddings"""
    descripcion = f"{row['nombre']} - Categoría: {row['categoria']}, Subcategoría: {row['subcategoria']}"
    descripcion += f", Carbohidratos: {row['carbohidratos']}g, Proteínas: {row['proteinas']}g"
    descripcion += f", Grasas: {row['grasas']}g, Fibra: {row['fibra']}g"
    descripcion += f", Índice Glucémico: {row['indiceGlicemico']}, Carga Glucémica: {row['cargaGlicemica']}"
    return descripcion

def preparar_base_alimentos():
    """Prepara la base de datos de alimentos para el sistema RAG"""
    global alimentos_embeddings_cache
    
    if df_alimentos.empty:
        print("⚠️ Base de datos de alimentos no disponible")
        return
    
    # Crear descripciones textuales de los alimentos
    alimentos_descripciones = df_alimentos.apply(crear_descripcion_alimento, axis=1).tolist()
    
    # Crear embeddings
    print(f"\n🧠 Creando sistema de recomendaciones nutricionales...")
    alimentos_embeddings_cache = crear_embeddings_alimentos(alimentos_descripciones, task_type="retrieval_document")
    
    if alimentos_embeddings_cache:
        print(f"✅ {len(alimentos_embeddings_cache)} embeddings de alimentos creados")
        
        # Guardar embeddings para uso futuro
        embeddings_file = os.path.join(personal_data_dir, "alimentos_embeddings.json")
        with open(embeddings_file, 'w') as f:
            json.dump({
                'embeddings': alimentos_embeddings_cache,
                'descripciones': alimentos_descripciones,
                'fecha': datetime.now().isoformat()
            }, f)
        print("✓ Sistema de recomendaciones guardado")
    else:
        print("❌ Falló creación del sistema de recomendaciones")

def filtrar_alimentos_por_perfil(perfil):
    """Filtra alimentos según el perfil del usuario"""
    alimentos_filtrados = df_alimentos.copy()
    
    # Filtros específicos para diabetes
    es_diabetico = perfil.get('esDiabetico', False)
    tipo_diabetes = perfil.get('tipoDiabetes', 'tipo2')
    
    if es_diabetico:
        print(f"🩺 Aplicando filtros para diabetes {tipo_diabetes}")
        
        if tipo_diabetes == 'tipo1':
            # Tipo 1: Control estricto, IG muy bajo preferible
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['indiceGlicemico'] <= 50]
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['cargaGlicemica'] <= 8]
            print("✓ Filtros aplicados: IG ≤ 50, CG ≤ 8 (Diabetes Tipo 1)")
            
        elif tipo_diabetes == 'tipo2':
            # Tipo 2: Control moderado, IG bajo-moderado
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['indiceGlicemico'] <= 55]
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['cargaGlicemica'] <= 10]
            print("✓ Filtros aplicados: IG ≤ 55, CG ≤ 10 (Diabetes Tipo 2)")
            
        elif tipo_diabetes == 'gestacional':
            # Gestacional: Control muy estricto
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['indiceGlicemico'] <= 45]
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['cargaGlicemica'] <= 8]
            print("✓ Filtros aplicados: IG ≤ 45, CG ≤ 8 (Diabetes Gestacional)")
            
        else:  # otro
            # Por defecto: criterios conservadores
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['indiceGlicemico'] <= 55]
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['cargaGlicemica'] <= 10]
            print("✓ Filtros aplicados: IG ≤ 55, CG ≤ 10 (Diabetes - criterios generales)")
    
    # Filtros básicos según restricciones (código existente)
    restricciones = perfil.get('restricciones', '').lower()
    
    if 'diabetes' in restricciones or 'diabetico' in restricciones:
        # Solo aplicar si no es diabético diagnosticado (evitar doble filtrado)
        if not es_diabetico:
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['indiceGlicemico'] <= 55]
            alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['cargaGlicemica'] <= 10]
    
    if 'vegetariano' in restricciones:
        # Excluir carnes y pescados
        alimentos_filtrados = alimentos_filtrados[
            ~alimentos_filtrados['subcategoria'].isin(['Aves', 'Carnes rojas', 'Carnes blancas', 'Pescados grasos', 'Pescados magros'])
        ]
    
    if 'sin lacteos' in restricciones or 'lactosa' in restricciones:
        # Excluir lácteos
        alimentos_filtrados = alimentos_filtrados[alimentos_filtrados['categoria'] != 'lacteos']
    
    return alimentos_filtrados

def encontrar_alimentos_recomendados(consulta, perfil, top_k=10):
    """Encuentra alimentos recomendados según consulta y perfil"""
    global alimentos_embeddings_cache
    
    if not consulta or not isinstance(consulta, str):
        print("❌ La consulta debe ser un string no vacío")
        return pd.DataFrame()
    
    if alimentos_embeddings_cache is None or df_alimentos.empty:
        print("⚠️ Sistema de recomendaciones no disponible")
        return pd.DataFrame()
    
    try:
        # Filtrar alimentos según perfil
        alimentos_filtrados = filtrar_alimentos_por_perfil(perfil)
        
        if alimentos_filtrados.empty:
            print("⚠️ No hay alimentos disponibles con las restricciones especificadas")
            return pd.DataFrame()
        
        # Crear embedding de la consulta
        consulta_embedding = crear_embeddings_alimentos([consulta], task_type="retrieval_query")
        
        if not consulta_embedding:
            print("❌ Falló generación de embedding de consulta")
            return pd.DataFrame()
        
        consulta_embedding = consulta_embedding[0]
        
        # Calcular similitudes solo para alimentos filtrados
        indices_filtrados = alimentos_filtrados.index.tolist()
        embeddings_filtrados = [alimentos_embeddings_cache[i] for i in indices_filtrados]
        
        embeddings_np = np.array(embeddings_filtrados)
        consulta_embedding_np = np.array(consulta_embedding)
        
        # Normalizar embeddings
        norm_consulta = np.linalg.norm(consulta_embedding_np)
        norm_alimentos = np.linalg.norm(embeddings_np, axis=1)
        
        if norm_consulta == 0 or np.any(norm_alimentos == 0):
            print("⚠️ Vector cero encontrado")
            return pd.DataFrame()
        
        # Calcular similitudes
        similitudes = np.dot(embeddings_np, consulta_embedding_np) / (norm_alimentos * norm_consulta)
        
        # Obtener top-K resultados
        top_k_indices_relativos = np.argsort(similitudes)[-top_k:]
        top_k_indices_absolutos = [indices_filtrados[i] for i in reversed(top_k_indices_relativos)]
        
        alimentos_recomendados = df_alimentos.loc[top_k_indices_absolutos].copy()
        alimentos_recomendados['similitud'] = [similitudes[i] for i in reversed(top_k_indices_relativos)]
        
        print(f"✅ {len(alimentos_recomendados)} alimentos recomendados encontrados")
        return alimentos_recomendados
        
    except Exception as e:
        print(f"❌ Error encontrando alimentos: {e}")
        return pd.DataFrame()

# Preparar el sistema de recomendaciones
if not df_alimentos.empty:
    preparar_base_alimentos()


🧠 Creando sistema de recomendaciones nutricionales...
✓ Generados 97 embeddings de alimentos
✅ 97 embeddings de alimentos creados
✓ Sistema de recomendaciones guardado
✓ Generados 97 embeddings de alimentos
✅ 97 embeddings de alimentos creados
✓ Sistema de recomendaciones guardado


In [197]:
# ========== 5. GENERADOR DE MEAL PLAN SEMANAL CON CONTROL GLUCÉMICO - JSON ESTRUCTURADO ==========

# Crear directorio de reportes
reportes_dir = os.path.join(personal_data_dir, "mis_reportes")
if not os.path.exists(reportes_dir):
    os.makedirs(reportes_dir)
    print(f"✓ Directorio de reportes creado: {reportes_dir}")

def cargar_meal_plans_anteriores():
    """Carga meal plans anteriores para evitar repeticiones"""
    meal_plans_anteriores = []
    
    try:
        # Buscar archivos de meal plans anteriores
        archivos_reportes = [f for f in os.listdir(reportes_dir) if f.startswith('meal_plan_') and f.endswith('.json')]
        
        # Ordenar por fecha (más recientes primero)
        archivos_reportes.sort(reverse=True)
        
        # Cargar últimos 4 meal plans para análisis
        for archivo in archivos_reportes[:4]:
            try:
                with open(os.path.join(reportes_dir, archivo), 'r', encoding='utf-8') as f:
                    meal_plan_data = json.load(f)
                    meal_plans_anteriores.append(meal_plan_data)
            except Exception as e:
                print(f"⚠️ Error cargando {archivo}: {e}")
        
        print(f"📚 Cargados {len(meal_plans_anteriores)} meal plans anteriores para análisis de variedad")
        return meal_plans_anteriores
        
    except Exception as e:
        print(f"📁 Primera vez generando meal plan: {e}")
        return []

def analizar_alimentos_usados_anteriormente(meal_plans_anteriores):
    """Analiza qué alimentos se han usado en meal plans anteriores"""
    alimentos_usados = {
        'desayunos': set(),
        'almuerzos': set(), 
        'cenas': set(),
        'snacks': set()
    }
    
    frecuencia_por_dia = {
        'lunes': set(),
        'martes': set(),
        'miercoles': set(),
        'jueves': set(),
        'viernes': set(),
        'sabado': set(),
        'domingo': set()
    }
    
    for meal_plan in meal_plans_anteriores:
        try:
            # Extraer alimentos usados por categoría
            if 'alimentos_utilizados' in meal_plan:
                alimentos_data = meal_plan['alimentos_utilizados']
                
                for categoria in ['desayunos', 'almuerzos', 'cenas', 'snacks']:
                    if categoria in alimentos_data:
                        alimentos_nombres = set(alimentos_data[categoria])
                        alimentos_usados[categoria].update(alimentos_nombres)
            
            # Extraer patrones por día de la semana
            if 'contenido_detallado' in meal_plan:
                contenido = meal_plan['contenido_detallado'].lower()
                for dia in frecuencia_por_dia.keys():
                    if dia in contenido:
                        # Buscar alimentos mencionados en este día
                        inicio_dia = contenido.find(dia)
                        siguiente_dia = None
                        
                        dias_orden = ['lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado', 'domingo']
                        try:
                            idx_actual = dias_orden.index(dia)
                            if idx_actual < len(dias_orden) - 1:
                                siguiente_dia = dias_orden[idx_actual + 1]
                                fin_dia = contenido.find(siguiente_dia, inicio_dia)
                            else:
                                fin_dia = len(contenido)
                        except:
                            fin_dia = inicio_dia + 500  # Buscar en próximos 500 caracteres
                        
                        if fin_dia == -1:
                            fin_dia = len(contenido)
                            
                        seccion_dia = contenido[inicio_dia:fin_dia]
                        
                        # Buscar nombres de alimentos en esta sección
                        for _, alimento in df_alimentos.iterrows():
                            nombre_lower = alimento['nombre'].lower()
                            if nombre_lower in seccion_dia:
                                frecuencia_por_dia[dia].add(alimento['nombre'])
                                
        except Exception as e:
            print(f"⚠️ Error analizando meal plan anterior: {e}")
    
    return alimentos_usados, frecuencia_por_dia

def filtrar_alimentos_para_variedad(alimentos_encontrados, categoria, alimentos_usados, factor_variedad=0.7):
    """Filtra alimentos para promover variedad"""
    if alimentos_encontrados.empty:
        return alimentos_encontrados
    
    alimentos_previos = alimentos_usados.get(categoria, set())
    
    if not alimentos_previos:
        return alimentos_encontrados
    
    # Separar alimentos en usados y no usados
    alimentos_nuevos = alimentos_encontrados[~alimentos_encontrados['nombre'].isin(alimentos_previos)]
    alimentos_repetidos = alimentos_encontrados[alimentos_encontrados['nombre'].isin(alimentos_previos)]
    
    # Calcular cuántos alimentos tomar de cada grupo
    total_necesarios = len(alimentos_encontrados)
    nuevos_necesarios = int(total_necesarios * factor_variedad)
    repetidos_necesarios = total_necesarios - nuevos_necesarios
    
    # Priorizar alimentos nuevos
    resultado = pd.DataFrame()
    
    if len(alimentos_nuevos) >= nuevos_necesarios:
        resultado = pd.concat([resultado, alimentos_nuevos.head(nuevos_necesarios)])
    else:
        resultado = pd.concat([resultado, alimentos_nuevos])
        repetidos_necesarios += (nuevos_necesarios - len(alimentos_nuevos))
    
    # Completar con alimentos repetidos si es necesario
    if repetidos_necesarios > 0 and not alimentos_repetidos.empty:
        resultado = pd.concat([resultado, alimentos_repetidos.head(repetidos_necesarios)])
    
    print(f"✨ Variedad para {categoria}: {len(alimentos_nuevos)} nuevos, {len(alimentos_repetidos)} repetidos")
    
    return resultado.reset_index(drop=True)

def clasificar_imc(imc):
    """Clasifica el IMC según categorías estándar"""
    if imc < 18.5:
        return "Bajo peso"
    elif imc < 25:
        return "Normal"
    elif imc < 30:
        return "Sobrepeso"
    else:
        return "Obesidad"

def extraer_hora(horario_str):
    """Extrae la hora en formato HH:MM de una cadena de fecha/hora"""
    try:
        if 'T' in horario_str:
            # Formato ISO con T
            time_part = horario_str.split('T')[1]
            return time_part[:5]  # HH:MM
        else:
            # Asumir que ya está en formato HH:MM
            return horario_str
    except:
        return "00:00"

def calcular_calorias_personalizadas(edad, peso, altura, genero, nivel_actividad):
    """Calcula las calorías diarias personalizadas usando fórmula Mifflin-St Jeor"""
    # TMB (Tasa Metabólica Basal)
    if genero.lower() == 'masculino':
        tmb = 10 * peso + 6.25 * altura - 5 * edad + 5
    else:
        tmb = 10 * peso + 6.25 * altura - 5 * edad - 161
    
    # Factor de actividad
    factores_actividad = {
        'sedentario': 1.2,
        'ligero': 1.375,
        'moderado': 1.55,
        'intenso': 1.725,
        'muy_intenso': 1.9
    }
    
    factor = factores_actividad.get(nivel_actividad, 1.375)
    return tmb * factor

def calcular_carbohidratos_objetivo(calorias_diarias, es_diabetico, tipo_diabetes):
    """Calcula los gramos de carbohidratos objetivo"""
    if es_diabetico:
        if tipo_diabetes == 'tipo1':
            # 35-40% de calorías de carbohidratos
            porcentaje = 0.375
        elif tipo_diabetes == 'tipo2':
            # 30-35% de calorías de carbohidratos
            porcentaje = 0.325
        else:
            # Conservador: 30%
            porcentaje = 0.30
    else:
        # Saludable: 45%
        porcentaje = 0.45
    
    # 1 gramo de carbo = 4 calorías
    return (calorias_diarias * porcentaje) / 4

def calcular_proteinas_objetivo(peso, nivel_actividad, edad):
    """Calcula los gramos de proteína objetivo"""
    # Base: 0.8g por kg de peso
    base = peso * 0.8
    
    # Ajuste por actividad
    if nivel_actividad == 'intenso':
        base *= 1.5
    elif nivel_actividad == 'moderado':
        base *= 1.2
    
    # Ajuste por edad (adultos mayores necesitan más)
    if edad > 65:
        base *= 1.2
    
    return base

def filtrar_alimentos_por_categoria_y_momento(categoria_deseada, momento_comida, es_diabetico=False, tipo_diabetes='tipo2'):
    """Filtra alimentos por categoría específica y momento de comida"""
    if df_alimentos.empty:
        return pd.DataFrame()
    
    # Filtrar por categoría principal solamente
    if categoria_deseada == "desayuno":
        # Para desayuno: proteínas, lácteos, carbohidratos, frutas
        categorias_permitidas = ['proteinas', 'lacteos', 'carbohidratos', 'frutas']
    elif categoria_deseada == "almuerzo":
        # Para almuerzo: proteínas, verduras, carbohidratos, grasas
        categorias_permitidas = ['proteinas', 'verduras', 'carbohidratos', 'grasas']
    elif categoria_deseada == "cena":
        # Para cena: proteínas magras, verduras, lácteos
        categorias_permitidas = ['proteinas', 'verduras', 'lacteos']
    elif categoria_deseada == "snack":
        # Para snacks: frutas, frutos secos, lácteos, verduras
        categorias_permitidas = ['frutas', 'grasas', 'lacteos', 'verduras']
    else:
        categorias_permitidas = ['proteinas', 'verduras', 'frutas', 'carbohidratos', 'lacteos', 'grasas']
    
    # Filtrar por categorías permitidas
    alimentos_filtrados = df_alimentos[df_alimentos['categoria'].isin(categorias_permitidas)].copy()
    
    # Aplicar filtros diabéticos si es necesario
    if es_diabetico:
        if tipo_diabetes == 'tipo1':
            # Control muy estricto
            alimentos_filtrados = alimentos_filtrados[
                (alimentos_filtrados['indiceGlicemico'] <= 50) & 
                (alimentos_filtrados['cargaGlicemica'] <= 8)
            ]
        elif tipo_diabetes == 'tipo2':
            # Control estricto
            alimentos_filtrados = alimentos_filtrados[
                (alimentos_filtrados['indiceGlicemico'] <= 55) & 
                (alimentos_filtrados['cargaGlicemica'] <= 10)
            ]
        else:
            # Control conservador
            alimentos_filtrados = alimentos_filtrados[
                (alimentos_filtrados['indiceGlicemico'] <= 50) & 
                (alimentos_filtrados['cargaGlicemica'] <= 10)
            ]
    
    return alimentos_filtrados.reset_index(drop=True)

def buscar_alimentos_inteligente(consulta, tipo_comida, perfil, top_k=15):
    """Búsqueda inteligente de alimentos combinando filtros y embedding"""
    
    # Determinar si es diabético
    es_diabetico = perfil.get('esDiabetico', False)
    tipo_diabetes = perfil.get('tipoDiabetes', 'tipo2')
    
    # Filtrar por categoría y momento de comida
    alimentos_por_categoria = filtrar_alimentos_por_categoria_y_momento(
        tipo_comida, tipo_comida, es_diabetico, tipo_diabetes
    )
    
    if alimentos_por_categoria.empty:
        print(f"⚠️ No se encontraron alimentos para {tipo_comida}")
        return pd.DataFrame()
    
    # Si tenemos embeddings, usar búsqueda semántica sobre los filtrados
    if alimentos_embeddings_cache:
        try:
            # Obtener índices de alimentos filtrados
            indices_filtrados = alimentos_por_categoria.index.tolist()
            
            # Crear embedding de la consulta
            consulta_embedding = crear_embeddings_alimentos([consulta], task_type="retrieval_query")
            
            if not consulta_embedding:
                print(f"⚠️ Error en embedding, usando filtros básicos para {tipo_comida}")
                return alimentos_por_categoria.head(top_k)
            
            consulta_embedding = consulta_embedding[0]
            
            # Calcular similitudes solo para alimentos filtrados
            embeddings_filtrados = []
            indices_validos = []
            
            for idx in indices_filtrados:
                if idx < len(alimentos_embeddings_cache):
                    embeddings_filtrados.append(alimentos_embeddings_cache[idx])
                    indices_validos.append(idx)
            
            if not embeddings_filtrados:
                return alimentos_por_categoria.head(top_k)
            
            embeddings_np = np.array(embeddings_filtrados)
            consulta_embedding_np = np.array(consulta_embedding)
            
            # Normalizar y calcular similitudes
            norm_consulta = np.linalg.norm(consulta_embedding_np)
            norm_alimentos = np.linalg.norm(embeddings_np, axis=1)
            
            if norm_consulta == 0 or np.any(norm_alimentos == 0):
                return alimentos_por_categoria.head(top_k)
            
            similitudes = np.dot(embeddings_np, consulta_embedding_np) / (norm_alimentos * norm_consulta)
            
            # Obtener top-K resultados
            top_k_indices_relativos = np.argsort(similitudes)[-top_k:]
            top_k_indices_absolutos = [indices_validos[i] for i in reversed(top_k_indices_relativos)]
            
            # Crear resultado con similitudes
            alimentos_resultado = df_alimentos.loc[top_k_indices_absolutos].copy()
            alimentos_resultado['similitud'] = [similitudes[i] for i in reversed(top_k_indices_relativos)]
            
            print(f"✅ {len(alimentos_resultado)} alimentos encontrados para {tipo_comida} con búsqueda semántica")
            return alimentos_resultado
            
        except Exception as e:
            print(f"⚠️ Error en búsqueda semántica para {tipo_comida}: {e}")
            return alimentos_por_categoria.head(top_k)
    else:
        # Sin embeddings, usar solo filtros
        print(f"✅ {len(alimentos_por_categoria)} alimentos encontrados para {tipo_comida} con filtros básicos")
        return alimentos_por_categoria.head(top_k)

def determinar_orden_optimo_alimentos(alimentos_lista):
    """Determina el orden óptimo de consumo según el algoritmo: Verduras → Proteínas → Grasas → Carbohidratos"""
    if not alimentos_lista:
        return []
    
    # Categorizar alimentos por tipo de macro
    verduras = []
    proteinas = []
    grasas = []
    carbohidratos = []
    otros = []
    
    for alimento in alimentos_lista:
        categoria = alimento.get('categoria', '').lower()
        
        if categoria == 'verduras':
            verduras.append(alimento)
        elif categoria == 'proteinas':
            proteinas.append(alimento)
        elif categoria == 'grasas':
            grasas.append(alimento)
        elif categoria == 'carbohidratos':
            carbohidratos.append(alimento)
        else:
            # Frutas, lácteos, etc. - clasificar por contenido nutricional principal
            carbs = alimento.get('carbohidratos', 0)
            prots = alimento.get('proteinas', 0)
            fats = alimento.get('grasas', 0)
            
            if prots > carbs and prots > fats:
                proteinas.append(alimento)
            elif fats > carbs and fats > prots:
                grasas.append(alimento)
            elif carbs > 5:  # Si tiene carbohidratos significativos
                carbohidratos.append(alimento)
            else:
                otros.append(alimento)
    
    # Construir orden óptimo
    orden_optimo = []
    
    # 1. Verduras primero
    if verduras:
        orden_optimo.extend(verduras)
    
    # 2. Proteínas segundo
    if proteinas:
        orden_optimo.extend(proteinas)
    
    # 3. Grasas saludables tercero
    if grasas:
        orden_optimo.extend(grasas)
    
    # 4. Carbohidratos al final
    if carbohidratos:
        orden_optimo.extend(carbohidratos)
    
    # 5. Otros alimentos al final
    if otros:
        orden_optimo.extend(otros)
    
    return orden_optimo

def crear_descripcion_comida_con_orden(alimentos_df, tipo_comida, alimentos_frecuentes):
    """Crea descripción detallada de alimentos organizados por orden óptimo de consumo"""
    if alimentos_df.empty:
        return f"No hay alimentos disponibles para {tipo_comida}"
    
    # Convertir DataFrame a lista de diccionarios
    alimentos_lista = []
    for _, alimento in alimentos_df.iterrows():
        alimentos_lista.append({
            'nombre': alimento['nombre'],
            'categoria': alimento['categoria'],
            'subcategoria': alimento['subcategoria'],
            'carbohidratos': alimento['carbohidratos'],
            'proteinas': alimento['proteinas'],
            'grasas': alimento['grasas'],
            'fibra': alimento['fibra'],
            'indiceGlicemico': alimento['indiceGlicemico'],
            'cargaGlicemica': alimento['cargaGlicemica']
        })
    
    # Obtener orden óptimo
    alimentos_ordenados = determinar_orden_optimo_alimentos(alimentos_lista)
    
    # Categorizar alimentos ordenados
    categorias_orden = {
        'verduras': [],
        'proteinas': [],
        'grasas': [],
        'carbohidratos': []
    }
    
    for alimento in alimentos_ordenados:
        categoria = alimento['categoria'].lower()
        if categoria == 'verduras':
            categorias_orden['verduras'].append(alimento)
        elif categoria == 'proteinas':
            categorias_orden['proteinas'].append(alimento)
        elif categoria == 'grasas':
            categorias_orden['grasas'].append(alimento)
        elif categoria == 'carbohidratos':
            categorias_orden['carbohidratos'].append(alimento)
        else:
            # Clasificar otros alimentos por contenido nutricional
            carbs = alimento['carbohidratos']
            prots = alimento['proteinas']
            fats = alimento['grasas']
            
            if prots > carbs and prots > fats:
                categorias_orden['proteinas'].append(alimento)
            elif fats > carbs and fats > prots:
                categorias_orden['grasas'].append(alimento)
            elif carbs > 5:
                categorias_orden['carbohidratos'].append(alimento)
            else:
                categorias_orden['verduras'].append(alimento)  # Por defecto en verduras
    
    # Crear descripción organizada
    descripcion_completa = []
    
    # Orden de presentación con emojis
    orden_presentacion = [
        ('verduras', '🥬 VERDURAS (Comer PRIMERO)', 1),
        ('proteinas', '🥩 PROTEÍNAS (Comer SEGUNDO)', 2),
        ('grasas', '🥑 GRASAS SALUDABLES (Comer TERCERO)', 3),
        ('carbohidratos', '🍞 CARBOHIDRATOS (Comer ÚLTIMO)', 4)
    ]
    
    for categoria_key, titulo, orden_num in orden_presentacion:
        alimentos_categoria = categorias_orden[categoria_key]
        
        if alimentos_categoria:
            descripcion_completa.append(f"\n**{titulo}:**")
            
            for alimento in alimentos_categoria:
                # Información básica con orden
                desc = f"• **{orden_num}️⃣ {alimento['nombre']}** (categoría: {alimento['categoria']})"
                
                # Información nutricional
                desc += f"\n  - Carbohidratos: {alimento['carbohidratos']}g"
                desc += f" | Proteínas: {alimento['proteinas']}g"
                desc += f" | Grasas: {alimento['grasas']}g"
                desc += f" | Fibra: {alimento['fibra']}g"
                
                # Información glucémica
                desc += f"\n  - IG: {alimento['indiceGlicemico']}"
                desc += f" | CG: {alimento['cargaGlicemica']:.1f}"
                
                # Clasificación glucémica
                if alimento['indiceGlicemico'] <= 35:
                    clasificacion_ig = "MUY BAJO"
                elif alimento['indiceGlicemico'] <= 50:
                    clasificacion_ig = "BAJO"
                elif alimento['indiceGlicemico'] <= 70:
                    clasificacion_ig = "MODERADO"
                else:
                    clasificacion_ig = "ALTO"
                
                desc += f" ({clasificacion_ig})"
                
                # Indicadores especiales - FAVORITOS CON ESTRELLA DESTACADA
                if alimento['nombre'] in alimentos_frecuentes:
                    desc += " ⭐ **FAVORITO DEL USUARIO** ⭐"
                
                if alimento['fibra'] >= 5:
                    desc += " 🌾 ALTA FIBRA"
                
                if categoria_key == 'proteinas' and alimento['grasas'] < 5:
                    desc += " 💪 PROTEÍNA MAGRA"
                
                if categoria_key == 'verduras':
                    desc += " 🥬 VEGETALES"
                
                descripcion_completa.append(desc)
    
    # Agregar información sobre el orden óptimo
    if len([cat for cat in categorias_orden.values() if cat]) > 1:
        descripcion_completa.insert(0, f"""
**🎯 ORDEN ÓPTIMO DE CONSUMO PARA {tipo_comida.upper()}:**
Seguir este orden puede reducir picos de glucosa hasta 37% según estudios clínicos:
1️⃣ Verduras PRIMERO → 2️⃣ Proteínas SEGUNDO → 3️⃣ Grasas TERCERO → 4️⃣ Carbohidratos ÚLTIMO
""")
    
    return "\n".join(descripcion_completa)

def guardar_meal_plan_como_reporte(meal_plan_texto, alimentos_desayuno, alimentos_almuerzo, alimentos_cena, alimentos_snacks, meal_plans_anteriores):
    """Guarda el meal plan como reporte completo"""
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Crear datos del reporte
        reporte_data = {
            'fecha_creacion': datetime.now().isoformat(),
            'usuario': mi_perfil.perfil.get('nombre', 'Usuario'),
            'contenido_meal_plan': meal_plan_texto,
            'alimentos_utilizados': {
                'desayunos': alimentos_desayuno['nombre'].tolist() if not alimentos_desayuno.empty else [],
                'almuerzos': alimentos_almuerzo['nombre'].tolist() if not alimentos_almuerzo.empty else [],
                'cenas': alimentos_cena['nombre'].tolist() if not alimentos_cena.empty else [],
                'snacks': alimentos_snacks['nombre'].tolist() if not alimentos_snacks.empty else []
            },
            'meal_plans_anteriores_count': len(meal_plans_anteriores),
            'perfil_usuario': mi_perfil.perfil
        }
        
        # Guardar JSON
        archivo_json = os.path.join(reportes_dir, f"meal_plan_{timestamp}.json")
        with open(archivo_json, 'w', encoding='utf-8') as f:
            json.dump(reporte_data, f, ensure_ascii=False, indent=2)
        
        # Guardar Markdown legible
        archivo_md = os.path.join(reportes_dir, f"meal_plan_{timestamp}.md")
        with open(archivo_md, 'w', encoding='utf-8') as f:
            f.write(meal_plan_texto)
        
        return archivo_json, archivo_md
        
    except Exception as e:
        print(f"⚠️ Error guardando reporte: {e}")
        return None, None

def crear_meal_plan_estructurado(alimentos_desayuno, alimentos_almuerzo, alimentos_cena, alimentos_snacks, perfil_usuario):
    """Crea un meal plan en formato JSON estructurado"""
    
    nombre = perfil_usuario.get('nombre', 'Usuario')
    es_diabetico = perfil_usuario.get('esDiabetico', False)
    tipo_diabetes = perfil_usuario.get('tipoDiabetes', 'tipo2')
    edad = perfil_usuario.get('edad', 25)
    peso = perfil_usuario.get('peso', 70)
    imc = perfil_usuario.get('imc', 25.0)
    
    # Horarios
    horario_desayuno = extraer_hora(perfil_usuario.get('horarioDesayuno', '07:30'))
    horario_comida = extraer_hora(perfil_usuario.get('horarioComida', '13:00'))
    horario_cena = extraer_hora(perfil_usuario.get('horarioCena', '19:30'))
    
    alimentos_frecuentes = perfil_usuario.get('alimentosFrecuentes', [])
    
    # Crear estructura base del meal plan
    meal_plan_estructura = {
        "metadatos": {
            "usuario": nombre,
            "fecha_generacion": datetime.now().isoformat(),
            "perfil": {
                "edad": edad,
                "peso": peso,
                "imc": round(imc, 1),
                "es_diabetico": es_diabetico,
                "tipo_diabetes": tipo_diabetes if es_diabetico else None,
                "nivel_control": "ESTRICTO" if es_diabetico else "PREVENTIVO"
            },
            "horarios": {
                "desayuno": horario_desayuno,
                "almuerzo": horario_comida,
                "cena": horario_cena
            },
            "alimentos_favoritos": alimentos_frecuentes
        },
        "dias": {},
        "resumen_semanal": {},
        "lista_compras": {},
        "tips_nutricionales": []
    }
    
    # Generar 7 días de comidas
    dias_semana = ['lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado', 'domingo']
    
    for i, dia in enumerate(dias_semana):
        meal_plan_estructura["dias"][dia] = crear_dia_estructurado(
            dia, i, alimentos_desayuno, alimentos_almuerzo, alimentos_cena, 
            alimentos_snacks, alimentos_frecuentes, horario_desayuno, 
            horario_comida, horario_cena
        )
    
    # Generar resumen semanal
    meal_plan_estructura["resumen_semanal"] = calcular_resumen_semanal(meal_plan_estructura["dias"])
    
    # Generar lista de compras
    meal_plan_estructura["lista_compras"] = generar_lista_compras_estructurada(meal_plan_estructura["dias"])
    
    # Generar tips nutricionales
    meal_plan_estructura["tips_nutricionales"] = generar_tips_estructurados(perfil_usuario)
    
    return meal_plan_estructura

def crear_dia_estructurado(dia, indice_dia, alimentos_desayuno, alimentos_almuerzo, 
                          alimentos_cena, alimentos_snacks, alimentos_favoritos,
                          horario_desayuno, horario_comida, horario_cena):
    """Crea la estructura de un día específico"""
    
    # Seleccionar alimentos para este día (rotación para variedad)
    desayuno_dia = seleccionar_alimentos_para_comida(alimentos_desayuno, indice_dia, 3, alimentos_favoritos)
    almuerzo_dia = seleccionar_alimentos_para_comida(alimentos_almuerzo, indice_dia, 4, alimentos_favoritos)
    cena_dia = seleccionar_alimentos_para_comida(alimentos_cena, indice_dia, 3, alimentos_favoritos)
    snack_matutino = seleccionar_alimentos_para_comida(alimentos_snacks, indice_dia, 1, alimentos_favoritos)
    merienda = seleccionar_alimentos_para_comida(alimentos_snacks, indice_dia + 3, 1, alimentos_favoritos)
    
    # Estructurar el día
    estructura_dia = {
        "desayuno": {
            "hora": horario_desayuno,
            "alimentos_en_orden": ordenar_alimentos_optimamente(desayuno_dia, alimentos_favoritos),
            "balance_nutricional": calcular_balance_comida(desayuno_dia)
        },
        "snack_matutino": {
            "hora": "10:00",
            "alimentos_en_orden": ordenar_alimentos_optimamente(snack_matutino, alimentos_favoritos),
            "balance_nutricional": calcular_balance_comida(snack_matutino)
        },
        "almuerzo": {
            "hora": horario_comida,
            "alimentos_en_orden": ordenar_alimentos_optimamente(almuerzo_dia, alimentos_favoritos),
            "balance_nutricional": calcular_balance_comida(almuerzo_dia)
        },
        "merienda": {
            "hora": "16:00",
            "alimentos_en_orden": ordenar_alimentos_optimamente(merienda, alimentos_favoritos),
            "balance_nutricional": calcular_balance_comida(merienda)
        },
        "cena": {
            "hora": horario_cena,
            "alimentos_en_orden": ordenar_alimentos_optimamente(cena_dia, alimentos_favoritos),
            "balance_nutricional": calcular_balance_comida(cena_dia)
        },
        "balance_diario": calcular_balance_diario([desayuno_dia, snack_matutino, almuerzo_dia, merienda, cena_dia])
    }
    
    return estructura_dia

def seleccionar_alimentos_para_comida(alimentos_disponibles, indice_dia, cantidad, alimentos_favoritos):
    """Selecciona alimentos para una comida específica priorizando favoritos"""
    if alimentos_disponibles.empty:
        return []
    
    # Separar favoritos y no favoritos
    favoritos = alimentos_disponibles[alimentos_disponibles['nombre'].isin(alimentos_favoritos)]
    no_favoritos = alimentos_disponibles[~alimentos_disponibles['nombre'].isin(alimentos_favoritos)]
    
    # Estrategia de selección con rotación
    seleccionados = []
    
    # Incluir al menos 1 favorito si está disponible
    if not favoritos.empty:
        idx_favorito = indice_dia % len(favoritos)
        seleccionados.append(favoritos.iloc[idx_favorito])
        cantidad -= 1
    
    # Completar con no favoritos
    if cantidad > 0 and not no_favoritos.empty:
        inicio = (indice_dia * 2) % len(no_favoritos)
        for i in range(cantidad):
            idx = (inicio + i) % len(no_favoritos)
            seleccionados.append(no_favoritos.iloc[idx])
    
    return seleccionados

def ordenar_alimentos_optimamente(alimentos_seleccionados, alimentos_favoritos):
    """Ordena los alimentos según el algoritmo óptimo: Verduras → Proteínas → Grasas → Carbohidratos"""
    
    orden_optimo = []
    
    # Categorizar por orden de consumo
    verduras = []
    proteinas = []
    grasas = []
    carbohidratos = []
    
    for alimento in alimentos_seleccionados:
        item_estructura = {
            "nombre": alimento['nombre'],
            "porcion_sugerida": calcular_porcion_sugerida(alimento),
            "orden_consumo": determinar_orden_consumo(alimento['categoria']),
            "es_favorito": alimento['nombre'] in alimentos_favoritos,
            "datos_nutricionales": {
                "carbohidratos": alimento['carbohidratos'],
                "proteinas": alimento['proteinas'],
                "grasas": alimento['grasas'],
                "fibra": alimento['fibra'],
                "indice_glucemico": alimento['indiceGlicemico'],
                "carga_glucemica": alimento['cargaGlicemica']
            }
        }
        
        # Clasificar por categoría para orden óptimo
        categoria = alimento['categoria'].lower()
        if categoria == 'verduras':
            verduras.append(item_estructura)
        elif categoria == 'proteinas':
            proteinas.append(item_estructura)
        elif categoria == 'grasas':
            grasas.append(item_estructura)
        elif categoria == 'carbohidratos':
            carbohidratos.append(item_estructura)
        else:
            # Clasificar otros por contenido nutricional principal
            if alimento['proteinas'] > alimento['carbohidratos']:
                proteinas.append(item_estructura)
            elif alimento['carbohidratos'] > 5:
                carbohidratos.append(item_estructura)
            else:
                verduras.append(item_estructura)
    
    # Construir orden óptimo
    orden_optimo.extend(verduras)
    orden_optimo.extend(proteinas)
    orden_optimo.extend(grasas)
    orden_optimo.extend(carbohidratos)
    
    return orden_optimo

def determinar_orden_consumo(categoria):
    """Determina el orden de consumo basado en la categoría"""
    orden_map = {
        'verduras': 1,
        'proteinas': 2,
        'grasas': 3,
        'carbohidratos': 4,
        'lacteos': 2,  # Generalmente proteicos
        'frutas': 4,   # Generalmente carbohidratos
        'bebidas': 1   # Pueden consumirse en cualquier momento
    }
    return orden_map.get(categoria.lower(), 3)

def calcular_porcion_sugerida(alimento):
    """Calcula una porción sugerida basada en el tipo de alimento"""
    categoria = alimento['categoria'].lower()
    
    porciones = {
        'verduras': "1-2 tazas",
        'proteinas': "150g" if alimento['subcategoria'] in ['Aves', 'Carnes rojas', 'Pescados magros'] else "1 taza",
        'carbohidratos': "1/2 taza" if alimento['indiceGlicemico'] > 55 else "3/4 taza",
        'lacteos': "150ml" if 'leche' in alimento['nombre'].lower() else "1 porción",
        'frutas': "1 mediana" if 'manzana' in alimento['nombre'].lower() else "1 taza",
        'grasas': "1 cucharada" if 'aceite' in alimento['nombre'].lower() else "30g"
    }
    
    return porciones.get(categoria, "1 porción")

def calcular_balance_comida(alimentos_comida):
    """Calcula el balance nutricional de una comida"""
    if not alimentos_comida:
        return {
            "carbohidratos_total": 0,
            "proteinas_total": 0,
            "grasas_total": 0,
            "fibra_total": 0,
            "carga_glucemica_total": 0
        }
    
    total_carbos = sum(alimento['carbohidratos'] for alimento in alimentos_comida)
    total_proteinas = sum(alimento['proteinas'] for alimento in alimentos_comida)
    total_grasas = sum(alimento['grasas'] for alimento in alimentos_comida)
    total_fibra = sum(alimento['fibra'] for alimento in alimentos_comida)
    total_cg = sum(alimento['cargaGlicemica'] for alimento in alimentos_comida)
    
    return {
        "carbohidratos_total": round(total_carbos, 1),
        "proteinas_total": round(total_proteinas, 1),
        "grasas_total": round(total_grasas, 1),
        "fibra_total": round(total_fibra, 1),
        "carga_glucemica_total": round(total_cg, 1)
    }

def calcular_balance_diario(comidas_del_dia):
    """Calcula el balance nutricional total del día"""
    todos_alimentos = []
    for comida in comidas_del_dia:
        todos_alimentos.extend(comida)
    
    balance = calcular_balance_comida(todos_alimentos)
    
    # Agregar evaluación
    es_saludable = (
        balance["carbohidratos_total"] <= 150 and
        balance["proteinas_total"] >= 50 and
        balance["fibra_total"] >= 20 and
        balance["carga_glucemica_total"] <= 80
    )
    
    balance["evaluacion"] = "✅ Dentro de objetivos" if es_saludable else "⚠️ Revisar porciones"
    
    return balance

def calcular_resumen_semanal(dias_meal_plan):
    """Calcula el resumen nutricional semanal"""
    totales_semanales = {
        "carbohidratos": 0,
        "proteinas": 0,
        "grasas": 0,
        "fibra": 0,
        "carga_glucemica": 0
    }
    
    for dia_data in dias_meal_plan.values():
        balance_diario = dia_data["balance_diario"]
        totales_semanales["carbohidratos"] += balance_diario["carbohidratos_total"]
        totales_semanales["proteinas"] += balance_diario["proteinas_total"]
        totales_semanales["grasas"] += balance_diario["grasas_total"]
        totales_semanales["fibra"] += balance_diario["fibra_total"]
        totales_semanales["carga_glucemica"] += balance_diario["carga_glucemica_total"]
    
    # Calcular promedios diarios
    promedios_diarios = {
        f"{macro}_promedio_diario": round(total / 7, 1)
        for macro, total in totales_semanales.items()
    }
    
    return {
        "totales_semanales": totales_semanales,
        "promedios_diarios": promedios_diarios,
        "beneficio_orden_optimo": "Reducción estimada de picos glucémicos: 25-37%",
        "cumplimiento_objetivos": evaluar_cumplimiento_objetivos(promedios_diarios)
    }

def evaluar_cumplimiento_objetivos(promedios):
    """Evalúa si se cumplen los objetivos nutricionales"""
    objetivos = {
        "carbohidratos": promedios["carbohidratos_promedio_diario"] <= 150,
        "proteinas": promedios["proteinas_promedio_diario"] >= 50,
        "fibra": promedios["fibra_promedio_diario"] >= 25,
        "carga_glucemica": promedios["carga_glucemica_promedio_diario"] <= 80
    }
    
    cumplidos = sum(objetivos.values())
    porcentaje = (cumplidos / len(objetivos)) * 100
    
    return {
        "objetivos_individuales": objetivos,
        "porcentaje_cumplimiento": round(porcentaje, 1),
        "estado_general": "Excelente" if porcentaje >= 90 else "Bueno" if porcentaje >= 75 else "Mejorable"
    }

def generar_lista_compras_estructurada(dias_meal_plan):
    """Genera una lista de compras organizada por categorías"""
    alimentos_necesarios = {}
    
    # Recopilar todos los alimentos de la semana
    for dia_data in dias_meal_plan.values():
        for comida_tipo in ['desayuno', 'snack_matutino', 'almuerzo', 'merienda', 'cena']:
            alimentos_comida = dia_data[comida_tipo]['alimentos_en_orden']
            for alimento in alimentos_comida:
                nombre = alimento['nombre']
                if nombre not in alimentos_necesarios:
                    alimentos_necesarios[nombre] = {
                        "categoria": determinar_categoria_compra(alimento),
                        "cantidad_estimada": 1,
                        "es_favorito": alimento['es_favorito']
                    }
                else:
                    alimentos_necesarios[nombre]["cantidad_estimada"] += 1
    
    # Organizar por categorías
    lista_por_categorias = {}
    for nombre, info in alimentos_necesarios.items():
        categoria = info["categoria"]
        if categoria not in lista_por_categorias:
            lista_por_categorias[categoria] = []
        
        lista_por_categorias[categoria].append({
            "alimento": nombre,
            "cantidad_semanal": info["cantidad_estimada"],
            "es_favorito": info["es_favorito"]
        })
    
    return lista_por_categorias

def determinar_categoria_compra(alimento_data):
    """Determina la categoría de compra para organizar la lista"""
    categoria_nutricional = alimento_data.get('datos_nutricionales', {})
    
    # Mapear categorías nutricionales a categorías de compra
    if categoria_nutricional:
        if alimento_data['orden_consumo'] == 1:
            return "🥬 Verduras y Vegetales"
        elif alimento_data['orden_consumo'] == 2:
            return "🥩 Proteínas"
        elif alimento_data['orden_consumo'] == 3:
            return "🥑 Grasas Saludables"
        else:
            return "🍞 Carbohidratos"
    
    return "🛒 Otros"

def generar_tips_estructurados(perfil_usuario):
    """Genera tips nutricionales personalizados en formato estructurado"""
    es_diabetico = perfil_usuario.get('esDiabetico', False)
    tipo_diabetes = perfil_usuario.get('tipoDiabetes', 'tipo2')
    
    tips = [
        {
            "categoria": "orden_optimo",
            "titulo": "Orden Óptimo de Consumo",
            "descripcion": "Come en este orden para reducir picos de glucosa",
            "instrucciones": [
                "1️⃣ Verduras PRIMERO (fibra y nutrientes)",
                "2️⃣ Proteínas SEGUNDO (saciedad y control)",
                "3️⃣ Grasas TERCERO (absorción lenta)",
                "4️⃣ Carbohidratos ÚLTIMO (menor impacto glucémico)"
            ]
        },
        {
            "categoria": "alimentos_favoritos",
            "titulo": "Tus Alimentos Favoritos",
            "descripcion": "Alimentos marcados con ⭐ en tu meal plan",
            "instrucciones": [f"• {alimento} - Incluido regularmente en tu plan" 
                            for alimento in perfil_usuario.get('alimentosFrecuentes', [])]
        }
    ]
    
    if es_diabetico:
        tips.append({
            "categoria": "control_diabetico",
            "titulo": f"Control Diabetes {tipo_diabetes.title()}",
            "descripcion": "Recomendaciones específicas para tu tipo de diabetes",
            "instrucciones": [
                "🩺 Consulta con tu médico sobre medicación",
                "📊 Monitorea glucosa antes y después de comidas",
                "⏰ Respeta horarios de comida establecidos",
                "🚶‍♀️ Camina 10-15 minutos después de comer",
                f"🎯 Mantén IG ≤ {55 if tipo_diabetes == 'tipo2' else 50}"
            ]
        })
    
    tips.append({
        "categoria": "beneficios_cientificos",
        "titulo": "Evidencia Científica",
        "descripcion": "Beneficios comprobados del orden alimentario",
        "instrucciones": [
            "📈 Reducción de picos glucémicos: 25-37%",
            "🍽️ Mayor saciedad y control de peso",
            "💊 Mejor efectividad de medicación diabética",
            "🧬 Optimización de respuesta insulínica"
        ]
    })
    
    return tips

def generar_meal_plan_semanal():
    """Genera un plan de comidas de 7 días en formato JSON estructurado"""
    nombre = mi_perfil.perfil.get('nombre', 'Usuario')
    
    print(f"\n🗓️ === GENERANDO MEAL PLAN ESTRUCTURADO PARA {nombre.upper()} ===")
    
    # Cargar meal plans anteriores para variedad
    meal_plans_anteriores = cargar_meal_plans_anteriores()
    alimentos_usados, frecuencia_por_dia = analizar_alimentos_usados_anteriormente(meal_plans_anteriores)
    
    # Buscar alimentos por categoría
    print("🔍 Buscando alimentos por categoría principal...")
    
    consulta_desayuno = f"desayuno proteínas carbohidratos lácteos"
    consulta_almuerzo = f"almuerzo proteínas verduras carbohidratos"
    consulta_cena = f"cena proteínas verduras lácteos"
    consulta_snacks = f"snack frutas lácteos verduras"
    
    alimentos_desayuno = buscar_alimentos_inteligente(consulta_desayuno, "desayuno", mi_perfil.perfil, 20)
    alimentos_almuerzo = buscar_alimentos_inteligente(consulta_almuerzo, "almuerzo", mi_perfil.perfil, 25)
    alimentos_cena = buscar_alimentos_inteligente(consulta_cena, "cena", mi_perfil.perfil, 20)
    alimentos_snacks = buscar_alimentos_inteligente(consulta_snacks, "snack", mi_perfil.perfil, 15)
    
    # Aplicar filtros de variedad
    alimentos_desayuno = filtrar_alimentos_para_variedad(alimentos_desayuno, 'desayunos', alimentos_usados, 0.8)
    alimentos_almuerzo = filtrar_alimentos_para_variedad(alimentos_almuerzo, 'almuerzos', alimentos_usados, 0.75)
    alimentos_cena = filtrar_alimentos_para_variedad(alimentos_cena, 'cenas', alimentos_usados, 0.8)
    alimentos_snacks = filtrar_alimentos_para_variedad(alimentos_snacks, 'snacks', alimentos_usados, 0.7)
    
    print("📊 Creando estructura JSON del meal plan...")
    
    # Crear meal plan estructurado
    meal_plan_json = crear_meal_plan_estructurado(
        alimentos_desayuno, alimentos_almuerzo, alimentos_cena, 
        alimentos_snacks, mi_perfil.perfil
    )
    
    return meal_plan_json, alimentos_desayuno, alimentos_almuerzo, alimentos_cena, alimentos_snacks, meal_plans_anteriores

def convertir_numpy_a_python(obj):
    """Convierte tipos NumPy a tipos nativos de Python para serialización JSON"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convertir_numpy_a_python(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convertir_numpy_a_python(item) for item in obj]
    elif hasattr(obj, 'item'):  # Para tipos NumPy escalares
        return obj.item()
    else:
        return obj

def guardar_meal_plan_json(meal_plan_json, alimentos_desayuno, alimentos_almuerzo, alimentos_cena, alimentos_snacks, meal_plans_anteriores):
    """Guarda el meal plan JSON estructurado"""
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Crear datos del reporte completo y convertir tipos NumPy
        reporte_completo = {
            "meal_plan": convertir_numpy_a_python(meal_plan_json),
            "alimentos_utilizados": {
                "desayunos": alimentos_desayuno['nombre'].tolist() if not alimentos_desayuno.empty else [],
                "almuerzos": alimentos_almuerzo['nombre'].tolist() if not alimentos_almuerzo.empty else [],
                "cenas": alimentos_cena['nombre'].tolist() if not alimentos_cena.empty else [],
                "snacks": alimentos_snacks['nombre'].tolist() if not alimentos_snacks.empty else []
            },
            "meal_plans_anteriores_count": len(meal_plans_anteriores)
        }
        
        # Guardar JSON estructurado
        archivo_json = os.path.join(reportes_dir, f"meal_plan_estructurado_{timestamp}.json")
        with open(archivo_json, 'w', encoding='utf-8') as f:
            json.dump(reporte_completo, f, ensure_ascii=False, indent=2)
        
        # Guardar también versión legible
        archivo_legible = os.path.join(reportes_dir, f"meal_plan_legible_{timestamp}.md")
        with open(archivo_legible, 'w', encoding='utf-8') as f:
            f.write(convertir_json_a_markdown_legible(meal_plan_json))
        
        return archivo_json, archivo_legible
        
    except Exception as e:
        print(f"⚠️ Error guardando meal plan JSON: {e}")
        return None, None

# EJECUTAR GENERACIÓN DEL MEAL PLAN ESTRUCTURADO
print("\n🚀 === GENERANDO MEAL PLAN JSON ESTRUCTURADO ===")
mi_meal_plan_json, alimentos_desayuno, alimentos_almuerzo, alimentos_cena, alimentos_snacks, meal_plans_anteriores = generar_meal_plan_semanal()

# Guardar como JSON estructurado
archivo_json, archivo_legible = guardar_meal_plan_json(mi_meal_plan_json, alimentos_desayuno, alimentos_almuerzo, alimentos_cena, alimentos_snacks, meal_plans_anteriores)

print(f"\n✅ ¡Meal Plan JSON estructurado creado!")
if archivo_json:
    print(f"📋 JSON estructurado: {archivo_json}")
    print(f"📄 Versión legible: {archivo_legible}")

# MOSTRAR MUESTRA DEL JSON EN EL NOTEBOOK
print("\n" + "="*80)
print("🔍 MUESTRA DEL MEAL PLAN JSON ESTRUCTURADO:")
print("="*80)

# Mostrar solo los primeros elementos para vista previa
muestra_json = {
    "metadatos": mi_meal_plan_json["metadatos"],
    "ejemplo_dia_lunes": mi_meal_plan_json["dias"]["lunes"],
    "resumen_semanal": mi_meal_plan_json["resumen_semanal"],
    "muestra_lista_compras": {k: v for i, (k, v) in enumerate(mi_meal_plan_json["lista_compras"].items()) if i < 2},
    "tips_nutricionales": mi_meal_plan_json["tips_nutricionales"][:2]
}

# Convertir tipos NumPy antes de serializar
muestra_json_serializable = convertir_numpy_a_python(muestra_json)

print(json.dumps(muestra_json_serializable, indent=2, ensure_ascii=False))

# Guardar la variable global para acceso posterior
meal_plan_json_global = mi_meal_plan_json


🚀 === GENERANDO MEAL PLAN JSON ESTRUCTURADO ===

🗓️ === GENERANDO MEAL PLAN ESTRUCTURADO PARA SANTIAGO ===
📚 Cargados 0 meal plans anteriores para análisis de variedad
🔍 Buscando alimentos por categoría principal...
✓ Generados 1 embeddings de alimentos
✅ 20 alimentos encontrados para desayuno con búsqueda semántica
✓ Generados 1 embeddings de alimentos
✅ 20 alimentos encontrados para desayuno con búsqueda semántica
✓ Generados 1 embeddings de alimentos
✅ 25 alimentos encontrados para almuerzo con búsqueda semántica
✓ Generados 1 embeddings de alimentos
✅ 25 alimentos encontrados para almuerzo con búsqueda semántica
✓ Generados 1 embeddings de alimentos
✅ 20 alimentos encontrados para cena con búsqueda semántica
✓ Generados 1 embeddings de alimentos
✅ 20 alimentos encontrados para cena con búsqueda semántica
✓ Generados 1 embeddings de alimentos
✅ 15 alimentos encontrados para snack con búsqueda semántica
📊 Creando estructura JSON del meal plan...

✅ ¡Meal Plan JSON estructurado cread